# Lab 8: Dynamic Prompts - Adaptando o Agente em Tempo Real

<img src="./assets/LC_DynamicPrompts.png" width="500">

## 🎯 O que você vai aprender neste Lab

**Dynamic Prompts** permitem adaptar o comportamento do agente em tempo real, baseado em:
- Contexto de runtime (quem é o usuário, permissões)
- Estado atual do agente
- Condições do ambiente

### Por que Dynamic Prompts são importantes?

1. **Personalização**: Comportamento diferente para diferentes usuários
2. **Segurança**: Restringir acesso baseado em permissões
3. **Contexto**: Adaptar a situações específicas
4. **Flexibilidade**: Um agente, múltiplos comportamentos

### O Problema com Prompts Estáticos:

```python
# Prompt estático - mesmo comportamento para todos
SYSTEM_PROMPT = "Você pode acessar todas as tabelas"

# Como restringir acesso para usuários não-admin? 😰
```

### Com Dynamic Prompts:

```python
# Prompt dinâmico - adapta ao contexto
def dynamic_prompt(request):
    if request.context.is_admin:
        return "Você pode acessar TODAS as tabelas"
    else:
        return "Você NÃO pode acessar tabelas sensíveis"
```

### Middleware no LangChain:

O LangChain oferece dois tipos de hooks:
1. **Node hooks**: Executam em pontos específicos do grafo
2. **Interceptor hooks**: Envolvem nós para modificar entrada/saída

`@dynamic_prompt` é um interceptor que modifica o prompt antes de cada chamada ao modelo.

## ⚙️ Setup - Configuração do Ambiente

In [ ]:
%pip install -U mlflow>=3 langchain>=1 langchain-community langgraph databricks-langchain --quiet
dbutils.library.restartPython()

In [ ]:
import mlflow
mlflow.langchain.autolog()

In [ ]:
# No Databricks, usamos spark.sql() diretamente
# Verifique as tabelas disponíveis no catálogo NASA GCN
%sql
SHOW TABLES IN nasa_gcn.silver

## 🔧 Definindo o Contexto de Runtime

Primeiro, definimos o que significa "contexto" para nosso agente. Neste caso, é simples: se o usuário é admin ou não.

O `RuntimeContext` pode conter qualquer informação útil:
- Dados do usuário (ID, nome, permissões)
- Configurações (idioma, fuso horário)
- Conexões (banco de dados, APIs)

In [ ]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_admin: bool  # Controla acesso a informações sensíveis

In [ ]:
from langchain_core.tools import tool

@tool
def execute_sql(query: str) -> str:
    """Execute a SQL command on Databricks and return results."""
    try:
        result = spark.sql(query).limit(5).toPandas()
        return result.to_markdown()
    except Exception as e:
        return f"Error: {e}"

### Template do System Prompt

Note o placeholder `{access_rules}` que será preenchido dinamicamente:

In [ ]:
SYSTEM_PROMPT_TEMPLATE = """Você é um analista cuidadoso de Databricks SQL especializado em dados da NASA GCN.

Regras:
- Pense passo a passo.
- Quando precisar de dados, chame a ferramenta `execute_sql` com UMA query SELECT.
- Apenas leitura; sem INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limite a 5 linhas de saída, a menos que o usuário peça mais.
{access_rules}
- Se a ferramenta retornar 'Error:', revise o SQL e tente novamente.
- Prefira listas explícitas de colunas; evite SELECT *.
- Use o catálogo nasa_gcn e schema silver (nasa_gcn.silver.tabela).
"""

## 🔄 Construindo o Dynamic Prompt

O decorador `@dynamic_prompt` transforma a função em um middleware.

### Como funciona:
1. A função recebe um `ModelRequest` com o contexto de runtime
2. Baseado no contexto (`is_admin`), gera regras diferentes
3. Retorna o prompt completo formatado

In [ ]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_admin:
        # Usuário não-admin: limita acesso a dados sensíveis de coordenadas
        access_rules = "- Usuário com acesso limitado. NÃO mostre colunas de coordenadas (ra, dec, ra_dec) - restritas a admins."
    else:
        access_rules = "- Você tem acesso de administrador. Todas as colunas estão disponíveis."

    return SYSTEM_PROMPT_TEMPLATE.format(access_rules=access_rules)

### Incluindo o middleware no agente

Passamos `middleware=[dynamic_system_prompt]` e `context_schema=RuntimeContext` para que o agente saiba usar nosso prompt dinâmico.

In [ ]:
from langchain.agents import create_agent
from databricks_langchain import ChatDatabricks

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.1,
)

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

## 🧪 Testando: Usuário NÃO Admin

Com `is_admin=False`, o agente NÃO deve mostrar coordenadas (ra, dec, ra_dec).

In [ ]:
# Teste com usuário NÃO admin - não deve mostrar coordenadas (ra, dec)
question = "Liste os 5 primeiros eventos com todos os detalhes disponíveis"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_admin=False),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 🧪 Testando: Usuário ADMIN

Com `is_admin=True`, o agente tem acesso total e pode mostrar coordenadas.

In [ ]:
# Teste com usuário ADMIN - pode ver todas as colunas incluindo coordenadas
question = "Liste os 5 primeiros eventos com todos os detalhes disponíveis"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_admin=True),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 📚 Resumo e Próximos Passos

### O que aprendemos:

| Conceito | Descrição |
|----------|-----------|
| **RuntimeContext** | Dados de contexto passados ao agente |
| **@dynamic_prompt** | Decorador para criar prompts dinâmicos |
| **middleware** | Lista de funções que modificam o comportamento |
| **context_schema** | Define a estrutura do contexto esperado |

### Casos de uso comuns:

1. **Controle de acesso**: Restringir dados por permissão
2. **Personalização**: Idioma, tom, formato por usuário
3. **Multi-tenant**: Comportamento diferente por cliente
4. **A/B testing**: Testar diferentes prompts

### Fluxo do Dynamic Prompt:

```
┌────────────────┐    ┌──────────────────┐    ┌─────────────┐
│ Context        │───▶│ @dynamic_prompt  │───▶│ LLM recebe  │
│ (is_admin=X)   │    │ Gera prompt      │    │ prompt      │
└────────────────┘    └──────────────────┘    └─────────────┘
```

### Próximo Lab:
No **Lab 9 - Human in the Loop**, você aprenderá a adicionar aprovação humana antes de executar ações críticas!